In [35]:
! cd ../data/retrotech && head signals.csv

import random
import pandas as pd
import numpy as np
import sys
sys.path.append('..')
from aips import *
from session_gen import SessionGenerator
import os
from IPython.core.display import display,HTML

"query_id","user","type","target","signal_time"
"u2_0_1","u2","query","nook","2019-07-31 08:49:07.3116"
"u2_1_2","u2","query","rca","2020-05-04 08:28:21.1848"
"u3_0_1","u3","query","macbook","2019-12-22 00:07:07.0152"
"u4_0_1","u4","query","Tv antenna","2019-08-22 23:45:54.1030"
"u5_0_1","u5","query","AC power cord","2019-10-20 08:27:00.1600"
"u6_0_1","u6","query","Watch The Throne","2019-09-18 11:59:53.7470"
"u7_0_1","u7","query","Camcorder","2020-02-25 13:02:29.3089"
"u9_0_1","u9","query","wireless headphones","2020-04-26 04:26:09.7198"
"u10_0_1","u10","query","Xbox","2019-09-13 16:26:12.0132"


In [36]:
def all_sessions():
    import glob
    return pd.concat([pd.read_csv(f, compression='gzip')
                      for f in glob.glob('*_sessions.gz')])

sessions = all_sessions()
sessions

,sess_id,query,rank,clicked_doc_id,clicked
0,2,dryer,0.0,84691226727,False
1,2,dryer,1.0,48231011402,False
2,2,dryer,2.0,12505382925,False
3,2,dryer,3.0,883049200057,False
4,2,dryer,4.0,74108056764,False
...,...,...,...,...,...
149995,55001,blue ray,25.0,22265004517,False
149996,55001,blue ray,26.0,885170038875,False
149997,55001,blue ray,27.0,786936817232,False
149998,55001,blue ray,28.0,600603132872,False


In [37]:
def fetch_products(doc_ids):
    import requests
    doc_ids = ["%s" % doc_id for doc_id in doc_ids]
    query = "upc:( " + " OR ".join(doc_ids) + " )"
    params = {'q':  query, 'wt': 'json', 'rows': len(doc_ids)}
    resp = requests.get('http://aips-solr:8983/solr/products/select', params=params)
    df = pd.DataFrame(resp.json()['response']['docs'])
    df['upc'] = df['upc'].astype('int64')
    
    df.insert(0, 'image', df.apply(lambda row: "<img height=\"100\" src=\"" + img_path_for_upc(row['upc']) + "\">", axis=1))
    
    return df

def render(judged, judgments_col='ctr', label=""):
    """ Render the computed judgments alongside the productns and description data"""
    w_prods = judged.merge(products, left_on='clicked_doc_id', right_on='upc', how='left')

    w_prods = w_prods[[judgments_col, 'image', 'upc', 'name', 'shortDescription']]

    return HTML(f"<h1>{label}</h1>" + w_prods.to_html(escape=False))

products = fetch_products(doc_ids=sessions['clicked_doc_id'].unique())
HTML(products.to_html(escape=False))

,image,upc,name,manufacturer,shortDescription,longDescription,id,_version_
0,,803238004525,Headphones - CD,Suicide Squeeze,\N,\N,c2490d46-dd98-4b55-a594-1e064893d3dd,1684550853904564231
1,,883393003458,"RCA - 46"" Class - LCD - 1080p - 60Hz - HDTV",RCA,\N,"This HDTV showcases stunning images up to 1080p and features a 4000:1 contrast ratio for an arresting viewing experience. The 3 HDMI inputs allow flexible connectivity options so you can enjoy your favorite movies, games and TV shows in high-definition.",953d1c22-88d7-4009-a5f6-b0783a411a11,1684550854019907593
2,,843404064434,ZAGG - InvisibleSHIELD for Apple® iPhone® 4 - Clear,ZAGG,Compatible with Apple® iPhone® 4; military-grade material,Protect your Apple iPhone 4's screen with this shield that provides invisible protection against scratches and dings without adding bulk.,8355f53e-da62-4667-a86e-9fca0f927ad4,1684550854146785281
3,,600603135101,Rocketfish™ - Protective Cover for NOOK,Rocketfish™,Compatible with NOOK WiFi and NOOK 3G+WiFi; thermoplastic polyurethane material,Protect your NOOK eReader from bumps and scratches with this lightweight cover made of eco-friendly thermoplastic polyurethane.,7346a389-ef33-448e-b7e8-45208226d43b,1684550854148882432
4,,813774010904,Samsung - Refurbished Wi-Fi Ready Blu-ray Player with HDMI Cable,Samsung,RefurbishedENERGY STAR QualifiedPlays DVD and Blu-ray Discs; 1080p video output; 2-channel audio outputLearn more about smart products.,"See movies come to life in brilliant high-definition with this Blu-ray player's 1080p video output. AllShare lets you enjoy music, video and photo files stored on your PC via the player. The included HDMI cable delivers stunning high-definition audio and video signals. Smart Blu-ray PlayerGet a world of instant entertainment with this smart Blu-ray player. Connect to the Internet and stream movies, listen to music and access a wide variety of other content on your HDTV. Learn more about smart products.NetflixPandoraYouTubeFlickrBlockbuster This product has been refurbished. Learn more.",fc81cd8c-376f-4f66-b4a2-59101009587f,1684550854156222466
5,,827396513927,Panasonic - Refurbished Wi-Fi Ready Blu-ray Disc Player,Panasonic,RefurbishedPlays DVD and Blu-ray Discs; 1080p upconversion; 2-channel analog output104.99,"Experience crystal-clear movies with this Blu-ray Disc player that features 1080p resolution for stunning visuals. BD Live provides access to a variety of real-time content including new previews, games, chat and more (network connection required). This product has been refurbished. Learn more.104.99",ff07ef55-4828-4604-8a37-890ee58ee827,1684550854167756803
6,,600603135088,Alpha - eReader Universal AC Charger,Alpha,Charge your eReader any way you need,"Your eReader is your own personal library tucked into your purse or pocket. The little device never fails to entertain, educate and fill you with wonder — unless, of course, it runs out of juice. Keep your pocket-sized book collection powered up and ready to go with the convenient eReader Universal AC Charger (not compatible with NOOK Color). When you want to recharge your eReader with an outlet instead of a computer, your options can be limited. But with this easy-to-use AC adapter, plugging your eReader into a wall is a snap. This charger package comes with a 1.5m USB-to-micro USB cable and a 1.5m USB-to-mini USB cable, to fit almost any device. These convenient cables connect your eReader to your computer, but can also connect to the included USB AC adapter, so you can tap into AC power from an outlet. It's your eReader. Keep it powered in a way that suits your lifestyle.",67ff73a9-4efc-4bdc-9257-c99f144e5ce6,1684550854177193984
7,,24543672067,Predators - Widescreen Dubbed Subtitle AC3 - Blu-ray Disc,\N,\N,\N,ea77d017-9392-4503-8cba-baab079a0e60,1684550854260031491
8,,97278016000,Furuno 600l Color LCD Fishfinder - DVD,\N,\N,\N,db3c0349-594f-4e89-85e3-8ceef1dc0494,1684550854261080072
9,,813580013571,M-Edge Accessories - Trip Jacket for NOOK - 

In [51]:
QUERY='dryer'

In [52]:
#dryer_sessions = dryer_sessions.merge(dryer_products, left_on='clicked_doc_id', right_on='upc', how='left')
dryer_sessions=sessions[sessions['query'] == QUERY]

num_sessions = len(dryer_sessions['sess_id'].unique())
dryer_ctrs = dryer_sessions.groupby('clicked_doc_id')['clicked'].sum().reset_index()
dryer_ctrs['ctr'] = dryer_ctrs['clicked'] / num_sessions
#dryer_ctrs = dryer_ctrs.sort_values('clicked', ascending=False)\
#                       .merge(products, left_on='clicked_doc_id', right_on='upc', how='left')
#dryer_sessions.groupby('clicked_doc_id').sum().sort_values('clicked', ascending=False)

render(dryer_ctrs.sort_values('ctr', ascending=False), judgments_col='ctr', label="Click-Thru-Rate Judgments")

,ctr,image,upc,name,shortDescription
0,0.3068,,84691226727,GE - 6.0 Cu. Ft. 3-Cycle Electric Dryer - White,Rotary electromechanical controls; 3 cycles; 3 heat selections; DuraDrum interior; Quiet-By-Design
1,0.2114,,48231011402,LG - 7.1 Cu. Ft. 7-Cycle Electric Dryer - White,Electronic controls with LED display; 7 cycles; Dial-A-Cycle option; sensor dry system; 5 temperature levels; 5 drying levels; NeveRust drum; LoDecibel quiet operation
2,0.0690,,74108096487,Conair - Infiniti Cord-Keeper Professional Tourmaline Ionic Hair Dryer - Fuchsia,Tourmaline ceramic technology; ionic technology; 1875 watts; Cool Shot function; 3 heat settings; 2 speed settings; 5' retractable cord; includes diffuser
3,0.0650,,12505525766,Smart Choice - 6' 30 Amp 3-Prong Dryer Cord,Heavy-duty PVC insulation; strain relief safety clamp
4,0.0572,,883049066905,Whirlpool - Affresh Washer Cleaner,Package include 3 tablets; removes and prevents odor-causing residue; compatible with all high-efficiency washers
5,0.0460,,74108056764,Conair - Infiniti Ionic Cord-Keeper Hair Dryer - Light Purple,1875 watts; dual voltage; 2 heat and speed settings
6,0.0442,,665331101927,Everything in Static - CD,\N
7,0.0442,,856751002097,Practecol - Dryer Balls (2-Pack),"Suitable for use on most dry cycles; reduces lint, static and wrinkles; improves heat circulation; 2-pack"
8,0.0408,,48231011396,LG - 3.5 Cu. Ft. 7-Cycle High-Efficiency Washer - White,ENERGY STAR QualifiedDigital controls; 7 cycles; SpeedWash cycle; 9 wash options; delay-wash; SenseClean system; 6Motion technology; TrueBalance antivibration system
9,0.0402,,74108007469,Conair - 1875-Watt Folding Handle Hair Dryer - Blue,2 heat/speed settings; cool shot button; dual voltage; professional-length line cord


## A typical session

What's a typical search session look like? Compute average rank per each product

In [53]:
dryer_sessions=sessions[sessions['query'] == QUERY]

typical_page = dryer_sessions.groupby('clicked_doc_id')['rank'].mean().reset_index().sort_values('rank')

render(typical_page, judgments_col='rank', label="A Typical Results Page")

,rank,image,upc,name,shortDescription
0,4.0460,,84691226727,GE - 6.0 Cu. Ft. 3-Cycle Electric Dryer - White,Rotary electromechanical controls; 3 cycles; 3 heat selections; DuraDrum interior; Quiet-By-Design
1,4.6998,,48231011402,LG - 7.1 Cu. Ft. 7-Cycle Electric Dryer - White,Electronic controls with LED display; 7 cycles; Dial-A-Cycle option; sensor dry system; 5 temperature levels; 5 drying levels; NeveRust drum; LoDecibel quiet operation
2,5.4270,,883049066905,Whirlpool - Affresh Washer Cleaner,Package include 3 tablets; removes and prevents odor-causing residue; compatible with all high-efficiency washers
3,6.2330,,36172950027,Tools in the Dryer: A Rarities Compilation - CD,\N
4,6.8746,,74108056764,Conair - Infiniti Ionic Cord-Keeper Hair Dryer - Light Purple,1875 watts; dual voltage; 2 heat and speed settings
5,7.6380,,74108096487,Conair - Infiniti Cord-Keeper Professional Tourmaline Ionic Hair Dryer - Fuchsia,Tourmaline ceramic technology; ionic technology; 1875 watts; Cool Shot function; 3 heat settings; 2 speed settings; 5' retractable cord; includes diffuser
6,8.3494,,12505525766,Smart Choice - 6' 30 Amp 3-Prong Dryer Cord,Heavy-duty PVC insulation; strain relief safety clamp
7,9.0478,,665331101927,Everything in Static - CD,\N
8,9.8750,,883929085118,A Charlie Brown Christmas - AC3 - Blu-ray Disc,\N
9,10.4744,,14381196320,The Mind Snatchers - DVD,\N


In [54]:
num_sessions = len(sessions['sess_id'].unique())
global_ctrs = sessions.groupby('rank')['clicked'].sum() / num_sessions
global_ctrs = global_ctrs.reset_index().rename(columns={'clicked': 'global_ctr'})
global_ctrs

,rank,global_ctr
0,0.0,0.237017
1,1.0,0.131233
2,2.0,0.075017
3,3.0,0.052133
4,4.0,0.049400
5,5.0,0.039050
6,6.0,0.029033
7,7.0,0.029817
8,8.0,0.016683
9,9.0,0.015167


In [55]:
#coec
coec_dryer_sums = dryer_sessions.groupby(['query', 'clicked_doc_id', 'rank'])['clicked'].sum()
coec_dryer_counts = dryer_sessions.groupby(['query', 'clicked_doc_id', 'rank'])['clicked'].count()

coecs = coec_dryer_sums / coec_dryer_counts
coecs = coecs.reset_index().rename(columns={'clicked': 'ctr_per_posn'})

coecs = coecs.merge(global_ctrs, on='rank', how='left')
coecs['ctr_over_global'] = coecs['ctr_per_posn'] / coecs['global_ctr']

coecs = coecs.groupby(['query', 'clicked_doc_id'])['ctr_over_global'].mean().reset_index()\
             .sort_values('ctr_over_global', ascending=False);

render(coecs, judgments_col='ctr_over_global', label="Clicks Over Expected Clicks")

,ctr_over_global,image,upc,name,shortDescription
0,2.793773,,48231011396,LG - 3.5 Cu. Ft. 7-Cycle High-Efficiency Washer - White,ENERGY STAR QualifiedDigital controls; 7 cycles; SpeedWash cycle; 9 wash options; delay-wash; SenseClean system; 6Motion technology; TrueBalance antivibration system
1,2.791186,,856751002097,Practecol - Dryer Balls (2-Pack),"Suitable for use on most dry cycles; reduces lint, static and wrinkles; improves heat circulation; 2-pack"
2,2.557776,,48231011402,LG - 7.1 Cu. Ft. 7-Cycle Electric Dryer - White,Electronic controls with LED display; 7 cycles; Dial-A-Cycle option; sensor dry system; 5 temperature levels; 5 drying levels; NeveRust drum; LoDecibel quiet operation
3,2.450566,,12505525766,Smart Choice - 6' 30 Amp 3-Prong Dryer Cord,Heavy-duty PVC insulation; strain relief safety clamp
4,2.058937,,74108096487,Conair - Infiniti Cord-Keeper Professional Tourmaline Ionic Hair Dryer - Fuchsia,Tourmaline ceramic technology; ionic technology; 1875 watts; Cool Shot function; 3 heat settings; 2 speed settings; 5' retractable cord; includes diffuser
5,1.997711,,84691226727,GE - 6.0 Cu. Ft. 3-Cycle Electric Dryer - White,Rotary electromechanical controls; 3 cycles; 3 heat selections; DuraDrum interior; Quiet-By-Design
6,1.884274,,74108007469,Conair - 1875-Watt Folding Handle Hair Dryer - Blue,2 heat/speed settings; cool shot button; dual voltage; professional-length line cord
7,1.863265,,84691226703,Hotpoint - 6.0 Cu. Ft. 3-Cycle Electric Dryer - White-on-White,Rotary controls; 3 cycles; 3 temperature settings; DuraDrum interior
8,1.841719,,36725578142,Samsung - 3.7 Cu. Ft. 9-Cycle Ultra Capacity Washer - White,ENERGY STAR QualifiedSoft dial touch pad; 9 cycles; sanitation/allergy cycle; 24-hour delay-start; Vibration Reduction Technology
9,1.833966,,85391137726,Cannonball Run II - DVD,\N


In [110]:
#sdbn
# For each session, find last click
# - everything above last click considered examined
# - everything clicked counted towards attractive
# - satisfaction is clicks / last clicks

last_clicks = dryer_sessions[dryer_sessions['clicked']].groupby(['sess_id', 'clicked'])['rank'].max().reset_index()
last_clicks = last_clicks.rename(columns={'rank': 'last_click_rank'})[['sess_id','last_click_rank']]
                            
# PROBLEM eliminate sessions without clicks!
sdbn = dryer_sessions.merge(last_clicks, on='sess_id', how='left')

sdbn['satisfied'] = sdbn.apply(lambda row: row['rank'] == row['last_click_rank'], axis=1)
sdbn['attractive'] = sdbn.apply(lambda row: row['rank'] <= row['last_click_rank'] and row['clicked'], axis=1)
sdbn['examined'] = sdbn.apply(lambda row: row['rank'] <= row['last_click_rank'], axis=1)



#    df.insert(0, 'image', df.apply(lambda row: "<img height=\"100\" src=\"" + img_path_for_upc(row['upc']) + "\">", axis=1))

In [111]:

sdbn[sdbn['sess_id'] == 3]

,sess_id,query,rank,clicked_doc_id,clicked,last_click_rank,satisfied,attractive,examined
30,3,dryer,0.0,84691226727,True,3.0,False,True,True
31,3,dryer,1.0,883049199061,False,3.0,False,False,True
32,3,dryer,2.0,883049066905,False,3.0,False,False,True
33,3,dryer,3.0,36172950027,True,3.0,True,True,True
34,3,dryer,4.0,74108056764,False,3.0,False,False,False
35,3,dryer,5.0,74108096487,False,3.0,False,False,False
36,3,dryer,6.0,12505525766,False,3.0,False,False,False
37,3,dryer,7.0,665331101927,False,3.0,False,False,False
38,3,dryer,8.0,883929085118,False,3.0,False,False,False
39,3,dryer,9.0,14381196320,False,3.0,False,False,False


In [112]:
type(last_clicks)

pandas.core.frame.DataFrame

In [113]:
# Clicks over examines
sdbn = sdbn[sdbn['examined']].groupby('clicked_doc_id').sum()

sdbn['attractiveness'] = sdbn['attractive'] / sdbn['examined']
sdbn['satisfaction'] = sdbn['satisfied'] / sdbn['examined']

sdbn
#sdbn = sdbn.sort_values('attractiveness', ascending=False)
#sdbn

,sess_id,rank,clicked,last_click_rank,satisfied,attractive,examined,attractiveness,satisfaction
clicked_doc_id,,,,,,,,,
12505382925,690129,2937.0,42.0,4797.0,34.0,42.0,283.0,0.148410,0.120141
12505451713,1941657,8856.0,44.0,13909.0,27.0,44.0,766.0,0.057441,0.035248
12505525476,748892,2714.0,43.0,4764.0,30.0,43.0,293.0,0.146758,0.102389
12505525766,3845495,9509.0,325.0,20720.0,211.0,325.0,1525.0,0.213115,0.138361
12505527456,1564144,8006.0,99.0,11753.0,76.0,99.0,641.0,0.154446,0.118565
14381196320,3014444,10159.0,179.0,18568.0,128.0,179.0,1206.0,0.148425,0.106136
17817263559,626316,2023.0,36.0,3672.0,29.0,36.0,249.0,0.144578,0.116466
36172950027,4765666,6544.0,201.0,21981.0,107.0,201.0,1873.0,0.107314,0.057128
36725560789,1284260,6467.0,64.0,9221.0,43.0,64.0,491.0,0.130346,0.087576


In [115]:
sdbn = sdbn.sort_values('attractiveness', ascending=False)
sdbn

,sess_id,rank,clicked,last_click_rank,satisfied,attractive,examined,attractiveness,satisfaction
clicked_doc_id,,,,,,,,,
84691226727,7717951,2267.0,1534.0,22790.0,582.0,1534.0,3064.0,0.500653,0.189948
48231011402,6526090,4516.0,1057.0,22881.0,554.0,1057.0,2590.0,0.408108,0.213900
883049199061,1047208,4665.0,109.0,6886.0,74.0,109.0,397.0,0.274559,0.186398
883049200057,866680,3663.0,91.0,5792.0,65.0,91.0,349.0,0.260745,0.186246
74108007469,2135097,9266.0,201.0,14573.0,140.0,201.0,846.0,0.237589,0.165485
85391137726,1334635,7003.0,117.0,9870.0,79.0,117.0,533.0,0.219512,0.148218
12505525766,3845495,9509.0,325.0,20720.0,211.0,325.0,1525.0,0.213115,0.138361
783722274422,1472648,7445.0,124.0,10671.0,81.0,124.0,583.0,0.212693,0.138937
48231011396,2382836,10131.0,204.0,16035.0,155.0,204.0,965.0,0.211399,0.160622


In [116]:
render(sdbn, judgments_col='attractiveness', label="Attractive Results")

,attractiveness,image,upc,name,shortDescription
0,0.500653,,84691226727,GE - 6.0 Cu. Ft. 3-Cycle Electric Dryer - White,Rotary electromechanical controls; 3 cycles; 3 heat selections; DuraDrum interior; Quiet-By-Design
1,0.408108,,48231011402,LG - 7.1 Cu. Ft. 7-Cycle Electric Dryer - White,Electronic controls with LED display; 7 cycles; Dial-A-Cycle option; sensor dry system; 5 temperature levels; 5 drying levels; NeveRust drum; LoDecibel quiet operation
2,0.274559,,883049199061,Maytag - 7.0 Cu. Ft. SuperSize Capacity Plus Electric Dryer - White,Electromechanical controls; 13 cycles; GentleBreeze drying system with IntelliDry sensor; powdercoat drum
3,0.260745,,883049200057,Whirlpool - 7.0 Cu. Ft. 13-Cycle Electric Dryer - White,Electromechanical controls; 13 automatic cycles; 4 temperature settings; powdercoat drum
4,0.237589,,74108007469,Conair - 1875-Watt Folding Handle Hair Dryer - Blue,2 heat/speed settings; cool shot button; dual voltage; professional-length line cord
5,0.219512,,85391137726,Cannonball Run II - DVD,\N
6,0.213115,,12505525766,Smart Choice - 6' 30 Amp 3-Prong Dryer Cord,Heavy-duty PVC insulation; strain relief safety clamp
7,0.212693,,783722274422,The Independent - Widescreen Subtitle - DVD,\N
8,0.211399,,48231011396,LG - 3.5 Cu. Ft. 7-Cycle High-Efficiency Washer - White,ENERGY STAR QualifiedDigital controls; 7 cycles; SpeedWash cycle; 9 wash options; delay-wash; SenseClean system; 6Motion technology; TrueBalance antivibration system
9,0.211281,,856751002097,Practecol - Dryer Balls (2-Pack),"Suitable for use on most dry cycles; reduces lint, static and wrinkles; improves heat circulation; 2-pack"
